In [63]:
import networkx as nx
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
adj_matrix = pd.read_csv("data/science.add9330_data_s1/Supplementary-Data-S1/all-all_connectivity_matrix.csv", index_col=0)

nodes = pd.read_csv("node_properties_clean.csv")

In [3]:
# Convert the adjacency matrix to an edge list
edge_list = adj_matrix.stack().reset_index().rename(columns={"level_0": "source", "level_1": "target", 0: "weight"})
# Filter out values where column "0" has value 0
edge_list = edge_list.loc[edge_list.weight != 0]
# Convert all to integer
edge_list = edge_list.astype(int)

In [42]:
# Get only the nodes that are in the edge list
filtered_nodes = nodes[nodes['skid'].isin(edge_list['source'])]

In [43]:
# Remove rows with NaN values
edge_list = edge_list.dropna().astype(int)

In [50]:
unique_celltypes = filtered_nodes['celltype'].unique()
colors = px.colors.qualitative.Alphabet[:len(unique_celltypes)]  # color sequence from plotly
# Create a dictionary mapping cell types to colors
celltype_to_color = dict(zip(unique_celltypes, colors))

# Add a new column to the nodes dataframe with the corresponding color for each cell type
filtered_nodes['color'] = filtered_nodes['celltype'].map(celltype_to_color)

C:\Users\eudald\AppData\Local\Temp\ipykernel_21572\215084946.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [65]:
import random

# Create a directed graph from the edge list
G = nx.from_pandas_edgelist(edge_list, 'source', 'target', ['weight'], create_using=nx.DiGraph())

# Add node properties
for node in filtered_nodes.itertuples():
    G.nodes[node.skid]['celltype'] = node.celltype

# Get a random subgraph of G
random_node = random.choice(list(G.nodes))
#sub_G = nx.ego_graph(G, random_node, radius=2)

# Get positions for the nodes in G
pos_ = nx.spring_layout(G, dim=3)

In [59]:
# Separate the x, y, and z coordinates of the node positions
pos_x = {node: pos for node, (pos, _, _) in pos_.items()}
pos_y = {node: pos for node, (_, pos, _) in pos_.items()}
pos_z = {node: pos for node, (_, _, pos) in pos_.items()}

# Add x, y, and z position to the nodes dataframe
filtered_nodes['x'] = filtered_nodes['skid'].map(pos_x)
filtered_nodes['y'] = filtered_nodes['skid'].map(pos_y)
filtered_nodes['z'] = filtered_nodes['skid'].map(pos_z)

# Create a dataframe of edges using source and target positions
edges_positions = edge_list.merge(filtered_nodes, left_on='source', right_on='skid', how='left')\
                       .rename(columns={'x': 'x0', 'y': 'y0'})\
                       .drop(columns='skid')\
                       .merge(filtered_nodes, left_on='target', right_on='skid', how='left')\
                       .rename(columns={'x': 'x1', 'y': 'y1'})\
                       .drop(columns='skid')\
                        .dropna()

# Prepare the edge traces for Plotly
edge_trace_x = np.empty(3*len(edges_positions), dtype=float)
edge_trace_x[::3] = edges_positions['x0']
edge_trace_x[1::3] = edges_positions['x1']
edge_trace_x[2::3] = None

edge_trace_y = np.empty(3*len(edges_positions), dtype=float)
edge_trace_y[::3] = edges_positions['y0']
edge_trace_y[1::3] = edges_positions['y1']
edge_trace_y[2::3] = None

# Prepare the edge traces for Plotly
edge_trace = go.Scatter3d(
    x=edge_trace_x,
    y=edge_trace_y,
    z=np.concatenate([filtered_nodes['z'], [None]*len(filtered_nodes)]),  # Add z coordinates
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines')

C:\Users\eudald\AppData\Local\Temp\ipykernel_21572\856690861.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\eudald\AppData\Local\Temp\ipykernel_21572\856690861.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\eudald\AppData\Local\Temp\ipykernel_21572\856690861.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [60]:
# Prepare the node traces for Plotly
node_trace = go.Scatter3d(
    x=filtered_nodes['x'],
    y=filtered_nodes['y'],
    z=filtered_nodes['z'],
    text='Cell type: ' + filtered_nodes['celltype'],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        color=filtered_nodes['color'],
        size=10,
        line=dict(width=2)))



In [61]:
# Create the interactive graph
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Network graph',
                    titlefont=dict(size=16),
                    showlegend=True,
                    hovermode='closest',
                    scene=dict(
                        xaxis=dict(range=[-2, 2]),  # Limit x-axis
                        yaxis=dict(range=[-2, 2]),  # Limit y-axis
                        zaxis=dict(range=[-2, 2])   # Limit z-axis
                    ),
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

fig.write_html("network.html")